# **Quinto conjunto de tareas a realizar**

## Paquetes necesarios e inicializaciones

La siguiente práctica consta de dos partes principales, la primera es entrenar una red neuronal para diferenciar características faciales y tras esto, crear un filtro que use las soluciones de la red, y la segunda, consiste en la realización de un filtro de ámbito libre.

Para la realización de las tareas será necesario realizar las siguientes instalaciones y creación de un nuevo enviroment

Se van a realizar las importaciones necesarias para la ejecución de los consiguientes fragmentos de código

In [3]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.utils import save_img
import pandas as pd

In [6]:
# --- 1. Configuración Inicial ---
IMG_SIZE = 224
ORIGINAL_DATA_PATH = "C:/Users/ivanp/Desktop/UTKFaces/" # Ruta a la carpeta original
ORGANIZED_DATA_DIR = "organized_data" # Carpeta para los datos divididos
BATCH_SIZE = 32

# --- Configuración del Entrenamiento en 2 Fases ---
EPOCHS_HEAD = 15  # Épocas solo para la capa final (congelada)
EPOCHS_TUNE = 50  # Épocas totales (Head + Tune)
LR_HEAD = 0.001   # Tasa de aprendizaje para la capa final
LR_TUNE = 1e-5    # Tasa de aprendizaje MUY baja para el ajuste fino

# --- 2. Lógica Condicional de Preparación de Datos ---

# COMPROBACIÓN: ¿Existen ya los datos organizados?
if not os.path.exists(ORGANIZED_DATA_DIR):
    print(f"Carpeta '{ORGANIZED_DATA_DIR}' no encontrada. Iniciando organización por primera vez...")
    
    # --- 2.1. Funciones de Carga (Solo para la primera vez) ---
    def get_label_from_age(age):
        if 0 <= age <= 29: return 0  # Joven
        elif 30 <= age <= 59: return 1  # Medio
        elif age >= 60: return 2  # Anciano
        return None

    def load_and_preprocess_data(path, img_size):
        images = []
        labels = []
        print("Iniciando carga de datos desde la carpeta original...")
        for filename in os.listdir(path):
            if not filename.endswith((".jpg", ".png", ".jpeg")):
                continue
            try:
                age_str = filename.split('_')[0]
                age = int(age_str)
                label = get_label_from_age(age)
                if label is not None:
                    full_path = os.path.join(path, filename)
                    img = tf.keras.utils.load_img(full_path, target_size=(img_size, img_size))
                    img_array = tf.keras.utils.img_to_array(img)
                    images.append(img_array)
                    labels.append(label)
            except Exception:
                pass
        print(f"Carga finalizada. Total de imágenes: {len(images)}")
        X = np.array(images) / 255.0
        y = np.array(labels)
        return X, y

    # --- 2.2. Guardar Imágenes Divididas (Solo para la primera vez) ---
    def save_split_data_to_folders(X_data, y_data, split_name, base_dir):
        class_names = ["joven", "medio", "anciano"]
        print(f"\nGuardando imágenes del conjunto: {split_name}...")
        for i, (img, label) in enumerate(zip(X_data, y_data)):
            class_name = class_names[label]
            target_dir = os.path.join(base_dir, split_name, class_name)
            if not os.path.exists(target_dir):
                os.makedirs(target_dir)
            filename = f"img_{i}.png"
            filepath = os.path.join(target_dir, filename)
            save_img(filepath, img * 255.0) # Guardar como 0-255
        print(f"Imágenes de {split_name} guardadas en {base_dir}/{split_name}")

    # --- 2.3. Ejecución de la Organización (Solo para la primera vez) ---
    X, y = load_and_preprocess_data(ORIGINAL_DATA_PATH, IMG_SIZE)
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)
    
    save_split_data_to_folders(X_train, y_train, "train", base_dir=ORGANIZED_DATA_DIR)
    save_split_data_to_folders(X_val, y_val, "validation", base_dir=ORGANIZED_DATA_DIR)
    save_split_data_to_folders(X_test, y_test, "test", base_dir=ORGANIZED_DATA_DIR)
    
    print("Organización de carpetas completa.")

else:
    print(f"Carpeta '{ORGANIZED_DATA_DIR}' encontrada. Cargando datos desde allí.")


# --- 3. Cargar Datos desde Carpetas Separadas (Usando tf.data) ---
# Esta sección ahora se ejecuta SIEMPRE.
train_dir = os.path.join(ORGANIZED_DATA_DIR, 'train')
val_dir = os.path.join(ORGANIZED_DATA_DIR, 'validation')
test_dir = os.path.join(ORGANIZED_DATA_DIR, 'test')

# Keras cargará los datos y automáticamente inferirá las etiquetas (0, 1, 2)
# desde los nombres de las subcarpetas (anciano, joven, medio)
train_dataset = image_dataset_from_directory(
    train_dir,
    seed=123,
    label_mode='int', # Etiquetas como 0, 1, 2
    class_names=['joven', 'medio', 'anciano'], # Asegurar el orden
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

val_dataset = image_dataset_from_directory(
    val_dir,
    seed=123,
    label_mode='int',
    class_names=['joven', 'medio', 'anciano'],
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

test_dataset = image_dataset_from_directory(
    test_dir,
    seed=123,
    label_mode='int',
    class_names=['joven', 'medio', 'anciano'],
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

# Optimización: Normalizar y preparar los datos para un alto rendimiento
def normalize(image, label):
    return tf.cast(image, tf.float32) / 255.0, label # Normalizar

train_dataset = train_dataset.map(normalize).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(normalize).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(normalize).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

print("Carga de datos desde carpetas finalizada y optimizada.")


# --- 4. Definir el Modelo (ResNet50) ---
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base_model.trainable = False # Empezamos con la base congelada

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)
output_layer = Dense(3, activation='softmax')(x) # 3 clases

model = Model(inputs=base_model.input, outputs=output_layer)


# --- 5. FASE 1: Compilar para Entrenamiento de "Head" ---
print("\n--- FASE 1: Compilando para entrenamiento de Capa Final (Head) ---")
model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=LR_HEAD),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
# model.summary()


# --- 5.5. Definir Callbacks ---
csv_log_file = 'training_log.csv'
csv_logger = CSVLogger(csv_log_file, separator=',', append=False)
print(f"\nResultados de cada época se guardarán en: {csv_log_file}")

best_model_file = 'best_age_model.keras'
model_checkpoint = ModelCheckpoint(
    filepath=best_model_file,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)
print(f"El mejor modelo (basado en val_accuracy) se guardará en: {best_model_file}")


# --- 6. FASE 1: Entrenar el Modelo (Solo "Head") ---
print(f"\n--- Iniciando FASE 1: Entrenamiento de Capa Final ({EPOCHS_HEAD} épocas) ---")

history_phase1 = model.fit(
    train_dataset, # Usamos el tf.data.Dataset
    epochs=EPOCHS_HEAD,
    validation_data=val_dataset, # Usamos el tf.data.Dataset
    callbacks=[csv_logger, model_checkpoint]
)

# --- 6.5. FASE 2: Preparar para Ajuste Fino (Fine-Tuning) ---
print(f"\n--- FASE 2: Preparando para Ajuste Fino (Fine-Tuning) ---")

base_model.trainable = True
print("Base del modelo descongelada.")

model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=LR_TUNE), # Tasa de aprendizaje BAJA
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

# Actualizar el CSVLogger para que AÑADA (append)
csv_logger.append = True

# --- 6.6. FASE 2: Continuar Entrenamiento (Fine-Tuning) ---
print(f"\n--- Iniciando FASE 2: Ajuste Fino (hasta la época {EPOCHS_TUNE}) ---")

initial_epoch_phase2 = history_phase1.epoch[-1] + 1

history_phase2 = model.fit(
    train_dataset, # Usamos el tf.data.Dataset
    epochs=EPOCHS_TUNE,
    initial_epoch=initial_epoch_phase2, # Empezar desde la época correcta
    validation_data=val_dataset, # Usamos el tf.data.Dataset
    callbacks=[csv_logger, model_checkpoint]
)

# --- 7. Evaluar el Modelo ---
print("\nEvaluando el MEJOR modelo guardado en el conjunto de prueba...")
print(f"Cargando el mejor modelo desde: {best_model_file}")
best_model = tf.keras.models.load_model(best_model_file)

test_loss, test_accuracy = best_model.evaluate(test_dataset) # Evaluar en el test_dataset
print(f"Precisión en el conjunto de prueba: {test_accuracy * 100:.2f}%")

# --- 8. Guardar historial final (para gráficas) ---
history_phase1_df = pd.DataFrame(history_phase1.history)
history_phase2_df = pd.DataFrame(history_phase2.history)
full_history_df = pd.concat([history_phase1_df, history_phase2_df], ignore_index=True)

summary_csv_file = 'training_history_summary.csv'
full_history_df.to_csv(summary_csv_file, index=False)
print(f"Resumen final del historial COMPLETO guardado en: {summary_csv_file}")

Carpeta 'organized_data' encontrada. Cargando datos desde allí.
Found 7095 files belonging to 3 classes.
Found 1521 files belonging to 3 classes.
Found 1521 files belonging to 3 classes.
Carga de datos desde carpetas finalizada y optimizada.

--- FASE 1: Compilando para entrenamiento de Capa Final (Head) ---

Resultados de cada época se guardarán en: training_log.csv
El mejor modelo (basado en val_accuracy) se guardará en: best_age_model.keras

--- Iniciando FASE 1: Entrenamiento de Capa Final (15 épocas) ---
Epoch 1/15
  4/222 ━━━━━━━━━━━━━━━━━━━━ 2:56 812ms/step - accuracy: 0.3783 - loss: 1.2354

KeyboardInterrupt: 

Tras lo anterior se procede a poner un filtro dependiendo de la edad de la persona detectada a través de la WebCam

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from retinaface import RetinaFace # Importar RetinaFace para detección de caras

# --- 1. Configuración Inicial ---
IMG_SIZE = 224 # Tamaño esperado por tu modelo Keras
MODEL_PATH = 'best_age_model.keras'
LABELS = {0: "JOVEN", 1: "MEDIO", 2: "ANCIANO"}
BOX_COLOR = (0, 255, 0) # Verde

# Rutas a tus imágenes de filtros PNG (¡Asegúrate de que existan!)
FILTER_PATHS = {
    0: 'bebe.png',    # Joven
    1: 'bigote.png',  # Medio
    2: 'baston.png'   # Anciano
}

# --- 2. Funciones Auxiliares ---

def load_filter_image(path):
    """Carga una imagen PNG con canal alfa."""
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if img is None:
        raise FileNotFoundError(f"No se pudo cargar el filtro: {path}")
    if img.shape[2] == 3: # Si es solo RGB, añadir canal alfa como completamente opaco
        b_channel, g_channel, r_channel = cv2.split(img)
        alpha_channel = np.ones(b_channel.shape, dtype=b_channel.dtype) * 255
        img = cv2.merge((b_channel, g_channel, r_channel, alpha_channel))
    return img

def overlay_image_alpha(img_base, img_overlay, x, y, alpha_mask=None):
    """
    Superpone una imagen con canal alfa sobre otra en una posición dada.
    img_base: Imagen de fondo (el fotograma de la cámara).
    img_overlay: Imagen a superponer (el filtro PNG con 4 canales RGBA).
    x, y: Coordenadas de la esquina superior izquierda donde colocar la imagen_overlay.
    alpha_mask: Si se proporciona, se usa esta máscara en lugar del canal alfa de img_overlay.
    """
    h, w = img_overlay.shape[0], img_overlay.shape[1]

    # Ajustar las coordenadas y dimensiones para que no excedan los límites de img_base
    y1, y2 = max(0, y), min(img_base.shape[0], y + h)
    x1, x2 = max(0, x), min(img_base.shape[1], x + w)

    y1_overlay, y2_overlay = max(0, -y), min(h, img_base.shape[0] - y)
    x1_overlay, x2_overlay = max(0, -x), min(w, img_base.shape[1] - x)

    # Si el filtro está completamente fuera del fotograma, no hacer nada
    if y1 >= y2 or x1 >= x2 or y1_overlay >= y2_overlay or x1_overlay >= x2_overlay:
        return img_base

    # Recortar la parte del filtro que realmente se superpondrá
    overlay_cropped = img_overlay[y1_overlay:y2_overlay, x1_overlay:x2_overlay]

    # Extraer el canal RGB y el canal alfa
    img_rgb = overlay_cropped[..., :3]
    alpha = overlay_cropped[..., 3:] / 255.0  # Normalizar a 0-1

    # Invertir el canal alfa
    alpha_inv = 1.0 - alpha

    # Recortar la región de la imagen base donde se superpondrá
    img_base_cropped = img_base[y1:y2, x1:x2]

    # Combinar las imágenes
    # fondo * (1 - alpha) + overlay * alpha
    img_base[y1:y2, x1:x2] = (img_base_cropped * alpha_inv + img_rgb * alpha).astype(np.uint8)
    return img_base


# --- 3. Cargar Modelos y Filtros ---
try:
    # Cargar tu modelo de clasificación de edad
    model_age = tf.keras.models.load_model(MODEL_PATH)
    print(f"Modelo {MODEL_PATH} cargado exitosamente.")
except Exception as e:
    print(f"Error cargando el modelo Keras: {e}")
    exit()

# Cargar todas las imágenes de filtro
loaded_filters = {}
print("Cargando imágenes de filtros...")
for age_id, path in FILTER_PATHS.items():
    try:
        loaded_filters[age_id] = load_filter_image(path)
        print(f"  Filtro '{path}' cargado para la edad {LABELS[age_id]}.")
    except FileNotFoundError as e:
        print(f"¡ADVERTENCIA! {e}. Este filtro no estará disponible.")
        loaded_filters[age_id] = None # Marcarlo como no disponible
    except Exception as e:
        print(f"Error desconocido cargando filtro '{path}': {e}. No disponible.")
        loaded_filters[age_id] = None

print("Iniciando cámara... Presiona 'q' para salir.")

# --- 4. Iniciar Captura de Video ---
cap = cv2.VideoCapture(0) # 0 para la cámara web por defecto

if not cap.isOpened():
    print("Error: No se pudo abrir la cámara web.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: No se pudo leer el fotograma.")
        break

    # Convertir a RGB para RetinaFace (OpenCV usa BGR por defecto)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    try:
        # --- 5. Detectar Caras con RetinaFace ---
        # RetinaFace devuelve un diccionario con una entrada por cada cara
        # Los resultados incluyen 'facial_areas' (bbox), 'landmarks', etc.
        faces_retina = RetinaFace.detect_faces(rgb_frame, threshold=0.9)

        if isinstance(faces_retina, dict): # Asegurarse de que se detectaron caras
            for identity in faces_retina.keys():
                face_info = faces_retina[identity]
                
                # Obtener el cuadro delimitador (bounding box)
                # RetinaFace da [x1, y1, x2, y2]
                x1, y1, x2, y2 = face_info['facial_area']
                x, y, w, h = x1, y1, (x2 - x1), (y2 - y1)

                # --- 6. Procesar cada Cara Detectada con tu Modelo Keras ---
                try:
                    # 1. Recortar la cara del fotograma original (en color)
                    face_roi = frame[y:y+h, x:x+w]

                    # 2. Pre-procesar la cara para tu modelo Keras
                    resized_face = cv2.resize(face_roi, (IMG_SIZE, IMG_SIZE))
                    normalized_face = resized_face / 255.0
                    input_face = np.expand_dims(normalized_face, axis=0)

                    # 3. Realizar la predicción de edad
                    prediction = model_age.predict(input_face, verbose=0)
                    
                    # 4. Obtener la clase de edad y la confianza
                    class_id = np.argmax(prediction[0])
                    confidence = np.max(prediction[0]) * 100
                    label_text = LABELS.get(class_id, "Desconocido")
                    
                    # 5. Formatear el texto para mostrar
                    text = f"{label_text} ({confidence:.1f}%)"

                    # 6. Dibujar en el fotograma original
                    cv2.rectangle(frame, (x, y), (x+w, y+h), BOX_COLOR, 2)
                    cv2.putText(
                        frame, 
                        text, 
                        (x, y - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.7, 
                        BOX_COLOR, 
                        2
                    )

                    # --- 7. Aplicar Filtro Visual ---
                    selected_filter = loaded_filters.get(class_id)
                    if selected_filter is not None:
                        # Redimensionar el filtro para que se ajuste a la cara
                        filter_resized = cv2.resize(selected_filter, (w, h)) 
                        frame = overlay_image_alpha(frame, filter_resized, x, y)
                
                except Exception as e:
                    print(f"Error procesando cara con modelo Keras: {e}")
                    pass # Continuar con otras caras o el siguiente fotograma

    except ValueError: # RetinaFace devuelve ValueError si no detecta caras
        pass # No se detectaron caras en este fotograma
    except Exception as e:
        print(f"Error general en la detección de RetinaFace: {e}")
        pass

    # --- 8. Mostrar el Resultado ---
    cv2.imshow('Detector de Edad con RetinaFace y Filtros', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# --- 9. Limpieza ---
cap.release()
cv2.destroyAllWindows()
print("Aplicación cerrada.")